In [1]:
# Install Pytorch & other libraries
# %pip install "torch==2.4.0" 
%pip install -q tensorboard pillow wandb 
 
# Install Hugging Face libraries
%pip install -q --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils"


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q pillow -U


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 문제 정의

과제물 사진과 과제물 정보를 주고 실제 정답을 맞추는 모델을 개발합니다. 

이 모델은 선생님들이 학생들의 수행평가를 채점한다고 가정합니다. 
본 프로젝트는 실제 교육 현장의 데이터와 차이가 있을 수 있으나, 자동 채점 시스템의 기본 원리와 가능성을 탐구하는 데 중점을 둡니다.

이번 예시에서는 Ko-SciecneQA 데이터셋을 사용할 건데요. 이 데이터셋은 12,726개의 아마존 제품의 제목, 이미지, 설명 및 메타데이터를 포함하고 있습니다.  
이 중에서 이미지를 가지고 있는 6,218개의 데이터 중 시간 절약을 위해서 모두 사용하지는 않고 여기서 20%(1,243)만 사용하겠습니다.

이미지, 문제, 힌트을 기반으로 정답을 생성하도록 모델을 파인튜닝하려 합니다.  
따라서 이미지, 문제, 힌트를 포함한 입력을 만들고, 이를 이용하여 정답을 찾아보겠습니다. 

In [3]:
# 참고: 이미지는 프롬프트에 직접 제공되지 않고 "processor"의 일부로 포함됨
prompt= """주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##CHOICES##: {korean_choices}
##HINT##: {korean_hint}"""

system_message = "당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다."

In [4]:
from datasets import load_dataset

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
   return {"messages": [
               {
                   "role": "system", # 시스템 역할
                   "content": [{"type": "text", "text": system_message}], # 시스템 메시지
               },
               {
                   "role": "user",  # 사용자 역할
                   "content": [
                       {
                           "type": "text",
                           # 제품명과 카테고리를 포함한 프롬프트 생성
                           "text": prompt.format(korean_choices=sample["korean_choices"], korean_hint=sample["korean_hint"]),
                       },{
                           "type": "image", # 이미지 타입
                           "image": sample["image"] if sample["image"] is not None else "", # 제품 이미지
                       }
                   ],
               },
               {
                   "role": "assistant", # AI 어시스턴트 역할
                   "content": [{"type": "text", "text": sample["answer_str"]}], # 제품 설명
               },
           ],
       }

# 허브에서 데이터셋 로드
dataset = load_dataset("daje/Ko-SciecneQA", split="train")
dataset = dataset.filter(lambda example: example["image"] is not None)
# 데이터셋을 OpenAI 메시지 형식으로 변환
# PIL.Image 타입을 유지하기 위해 리스트 컴프리헨션 사용 (.map()은 이미지를 바이트로 변환해버림)
dataset = [format_data(sample) for sample in dataset]

print(dataset[345]["messages"])

README.md:   0%|          | 0.00/881 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/412M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12726 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12726 [00:00<?, ? examples/s]

[{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##CHOICES##: ['신시내티', '링컨', '클리블랜드', '콜럼버스']\n##HINT##: "}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7E6F6A1C6450>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '콜럼버스'}]}]


In [5]:
len(dataset), dataset[345]

(6218,
 {'messages': [{'role': 'system',
    'content': [{'type': 'text',
      'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]},
   {'role': 'user',
    'content': [{'type': 'text',
      'text': "주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##CHOICES##: ['신시내티', '링컨', '클리블랜드', '콜럼버스']\n##HINT##: "},
     {'type': 'image',
      'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429>}]},
   {'role': 'assistant', 'content': [{'type': 'text', 'text': '콜럼버스'}]}]})

In [6]:
# 위에 준 코드 실행 후
train_dataset = dataset[:int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

print(f"데이터 개수: {len(dataset)}")

데이터 개수: 6218


In [7]:
len(train_dataset), len(test_dataset)

(5596, 622)

## trl의 SFTTrainer를 이용한 파인 튜닝

`trl`의 SFTTrainer를 사용해 모델을 파인튜닝할 건데요. SFTTrainer는 오픈소스 LLM과 VLM의 지도 파인튜닝을 매우 간단하게 만들어줍니다.  
SFTTrainer는 `transformers` 라이브러리의 `Trainer`를 상속받아서 로깅, 평가, 체크포인트 등 모든 기능을 지원하면서도 추가적인 편의 기능을 제공합니다.

이번 예시에서는 PEFT 기능을 사용할 예정입니다. PEFT 방법으로는 QLoRA를 사용할 건데, 이는 양자화와 LoRA 튜닝을 같이 사용하여 대규모 언어 모델의 메모리 사용량을 줄이는 기술입니다.

* 참고: 멀티모달 입력에 패딩이 필요하기 때문에 Flash Attention은 사용할 수 없습니다.*

Qwen 2 VL 7B 모델을 사용할 예정이지만, `model_id` 변수만 바꾸면 Meta AI의 Llama-3.2-11B-Vision, Mistral AI의 Pixtral-12B 등 다른 모델로도 쉽게 교체할 수 있습니다. bitsandbytes를 사용해 모델을 4비트로 양자화할 예정입니다.

* 참고: 모델이 클수록 더 많은 메모리가 필요합니다. 이번 예시에서는 7B 모델을 사용할 예정입니다.*

VLM 학습을 위해 LLM, 토크나이저, 프로세서를 올바르게 준비하는 것이 매우 중요합니다. 프로세서는 특수 토큰과 이미지를 입력에 포함시키는 역할을 담당하는 모듈입니다.

In [8]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-72B-Instruct" 

# BitsAndBytes 4비트 양자화 설정
# bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,                             # 4비트 양자화 사용
#    bnb_4bit_use_double_quant=True,               # 이중 양자화 사용으로 메모리 추가 절약
#    bnb_4bit_quant_type="nf4",                    # 4비트 양자화 타입 설정(normalized float 4)
#    bnb_4bit_compute_dtype=torch.bfloat16         # 연산 시 bfloat16 타입 사용
# )

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   # attn_implementation="flash_attention_2",     # 학습시에는 flash attention 2 미지원
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
   # quantization_config=bnb_config                # 위에서 정의한 양자화 설정 적용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/108k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

다음은 QWEN의 템플릿 예시입니다.  

```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|im_end|>
<|im_start|>assistant
거대 언어 모델의 답변<|im_end|>
```

멀티모달 QWEN은 이렇게 사용할 겁니다.

```python
<|im_start|>system
시스템 프롬프트<|im_end|>
<|im_start|>user
사용자의 질문<|vision_start|>이미지<|vision_end|><|im_end|>
<|im_start|>assistant
거대 언어 모델의 답변<|im_end|>
```

In [9]:
# Preparation for inference
text = processor.apply_chat_template(
    dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.<|im_end|>
<|im_start|>user
주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##CHOICES##: ['Kathleen의 스노보드가 왁스 층이 있을 때와 없을 때 중 어느 경우에 언덕을 더 빨리 내려오는가?', 'Kathleen의 스노보드가 얇은 왁스 층이 있을 때와 두꺼운 왁스 층이 있을 때 중 어느 경우에 언덕을 더 빨리 내려오는가?']
##HINT##: 아래의 글은 실험을 설명합니다. 글을 읽고 아래의 지시를 따르세요.

Kathleen은 스노보드의 밑면에 얇은 왁스 층을 바르고 언덕을 곧장 내려갔습니다. 그런 다음, 왁스를 제거하고 다시 스노보드를 타고 언덕을 내려갔습니다. 그녀는 왁스를 바른 상태와 바르지 않은 상태를 번갈아 가며 네 번 더 반복했습니다. 그녀의 친구 Bryant는 각 타이밍을 측정했습니다. Kathleen과 Bryant는 왁스를 바른 스노보드로 언덕을 내려가는 평균 시간과 왁스를 바르지 않은 스노보드로 내려가는 평균 시간을 계산했습니다.
그림: 언덕을 내려가는 스노보드.<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
Kathleen의 스노보드가 왁스 층이 있을 때와 없을 때 중 어느 경우에 언덕을 더 빨리 내려오는가?<|im_end|>



SFTTrainer는 peft와 기본적으로 통합되어 있어 LoraConfig를 만들어서 트레이너에 제공하기만 하면 됩니다.

In [10]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules=[
            "q_proj",
            "up_proj",
            "o_proj",
            "k_proj",
            "down_proj",
            "gate_proj",
            "v_proj"],
        task_type="CAUSAL_LM",
)

학습을 시작하기 전에 사용할 하이퍼파라미터(SFTConfig)를 정의하고 입력이 모델에 올바르게 제공되는지 확인해야 합니다.  

텍스트만 사용하는 지도 파인튜닝과 달리 모델에 이미지도 함께 제공해야 하는데요. 이를 위해 입력을 올바르게 포맷팅하고 이미지 특징을 포함하는 커스텀 DataCollator를 만들어야 합니다.  

Qwen2 팀이 제공하는 유틸리티 패키지의 process_vision_info 메서드를 사용할 예정입니다. Llama 3.2 Vision 같은 다른 모델을 사용하는 경우라면, 동일한 방식으로 이미지 정보가 처리되는지 확인해봐야 합니다.

In [11]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

# SFTConfig를 통해 학습 설정을 정의
args = SFTConfig(
    output_dir="qwen2-72b-instruct-homeworks",          # 학습된 모델과 체크포인트를 저장할 디렉터리 경로 및 리포지토리 ID
    num_train_epochs=3,                                # 전체 학습 에포크 수 (데이터셋을 몇 번 반복할지 설정)
    per_device_train_batch_size=1,                     # 각 장비(GPU)당 사용될 배치 사이즈 (메모리와 연관됨)
    gradient_accumulation_steps=8,                     # 경사 누적 스텝 수 (이 횟수만큼 기울기를 누적한 후 업데이트)
    gradient_checkpointing=True,                       # 메모리 절약을 위한 gradient checkpointing 활성화 (메모리 최적화)
    optim="adamw_torch_fused",                         # AdamW 옵티마이저 (fused 버전 사용으로 학습 속도 향상)
    logging_steps=5,                                   # 몇 스텝마다 로그를 출력할지 설정 (여기선 5 스텝마다 로그)
    save_strategy="epoch",                             # 매 에포크마다 체크포인트 저장 설정
    learning_rate=1e-5,                                # 학습률 (QLoRA 논문에서 추천된 값 사용)
    bf16=True,                                         # bfloat16 정밀도 사용 (메모리 절약 및 속도 향상)
    tf32=True,                                         # tf32 정밀도 사용 (NVIDIA GPU에서 학습 속도 향상)
    max_grad_norm=0.3,                                 # 기울기 클리핑을 위한 최대 기울기 값 (QLoRA 논문에서 추천된 값)
    warmup_ratio=0.03,                                 # 학습 초기에 학습률을 점진적으로 올리는 warmup 비율 (QLoRA 논문에서 추천된 값)
    lr_scheduler_type="constant",                      # 일정한 학습률 스케줄러 사용 (학습률이 변하지 않음)
    # push_to_hub=True,                                  # 학습된 모델을 Hugging Face Hub에 푸시할지 여부
    report_to="tensorboard",                           # TensorBoard를 통해 학습 상태를 모니터링
    gradient_checkpointing_kwargs={"use_reentrant": False}, # reentrant gradient checkpointing 설정 (비재진입 방식 사용)
    dataset_text_field="",                             # 데이터셋에서 텍스트 필드를 위한 더미 필드 (collator에서 필요)
    dataset_kwargs={"skip_prepare_dataset": True}      # collator에서 데이터셋 전처리를 건너뛰기 위한 설정
)

# 불필요한 열 삭제하지 않도록 설정 (학습 중 사용되지 않는 열이라도 유지)
args.remove_unused_columns = False

In [12]:
# 텍스트와 이미지 쌍을 인코딩하기 위한 데이터 collator 함수 정의
def collate_fn(examples):
    # 각 예제에서 텍스트와 이미지를 추출하고, 텍스트는 채팅 템플릿을 적용
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

    # 텍스트를 토크나이징하고 이미지를 처리하여 일괄 처리(batch) 형태로 변환
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

    # labels로 사용할 input_ids 복사본 생성 후, 패딩 토큰을 -100으로 설정하여 손실 계산 시 무시하도록 함
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  # 패딩 토큰 손실 계산 제외

    # 특정 이미지 토큰 인덱스는 손실 계산에서 무시 (모델에 따라 다름)
    if isinstance(processor, Qwen2VLProcessor):  
        # Qwen2VL 모델의 이미지 토큰 인덱스
        image_tokens = [151652, 151653, 151655]
    else:
        # 다른 모델에서 이미지 토큰 ID를 얻어 손실 계산에서 제외
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    
    # 손실 계산 시 이미지 토큰 인덱스를 무시하도록 설정
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    
    # 배치에 labels 추가 (손실 계산 시 사용)
    batch["labels"] = labels

    return batch

In [13]:
# 단일 예시 확인
example = dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. \n\n##CHOICES##: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']\n##HINT##: "}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x429 at 0x7E6F6A1B3750>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '웨스트버지니아'}]}]}


In [14]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 533])
어텐션 마스크 형태: torch.Size([1, 533])
이미지 픽셀 형태: torch.Size([1620, 1176])
레이블 형태: torch.Size([1, 533])


In [15]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        126674,  18411,  63332,  34395, 126674,  18411, 131417, 132526,  16560,
         15235,  21388,  78952,     13, 151645,    198, 151644,    872,    198,
         54330,  31079,  85251,  90667,  21329,  80573,   7704,   1143,     46,
         15835,    565,     11,   7704,     39,   3221,    565,  18411,  63332,
         34395,  36055, 132760,  17877, 131417, 132526,  16560,  69192,    247,
         37087, 129392,  40281,  56039,  78952,     13, 130887, 135968, 126550,
         23573,  36055, 132760,  17877, 131417, 132526,  50302,     13,   4710,
           565,  30498,  15835,    565,     25,   2509, 144025,  53189,  79004,
         21329,  83036,  52959,    516,    364, 126746,  12802,  21329, 126898,
         60315,    516,    364, 126898,  28002,  92817,  60315,    516,    364,
         34992,  44680,    223,    112,  50340,  47324, 125544,   4432,    565,
            39,   3221

In [16]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        126674,  18411,  63332,  34395, 126674,  18411, 131417, 132526,  16560,
         15235,  21388,  78952,     13, 151645,    198, 151644,    872,    198,
         54330,  31079,  85251,  90667,  21329,  80573,   7704,   1143,     46,
         15835,    565,     11,   7704,     39,   3221,    565,  18411,  63332,
         34395,  36055, 132760,  17877, 131417, 132526,  16560,  69192,    247,
         37087, 129392,  40281,  56039,  78952,     13, 130887, 135968, 126550,
         23573,  36055, 132760,  17877, 131417, 132526,  50302,     13,   4710,
           565,  30498,  15835,    565,     25,   2509, 144025,  53189,  79004,
         21329,  83036,  52959,    516,    364, 126746,  12802,  21329, 126898,
         60315,    516,    364, 126898,  28002,  92817,  60315,    516,    364,
         34992,  44680,    223,    112,  50340,  47324, 125544,   4432,    565,
            39,   322

In [17]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 문제를 보고 문제를 맞추는 AI Assistant입니다.<|im_end|>
<|im_start|>user
주어진 이미지와 ##ChOICES##, ##HINT##를 보고 정답을 맞추는 숙제 도우미입니다. 가장 적절한 정답을 맞추세요. 

##CHOICES##: ['웨스트버지니아', '루이지애나', '애리조나', '오클라호마']
##HINT##: <|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad

In [18]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    dataset_text_field="",
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

Trainer 인스턴스의 train() 메서드를 호출하여 모델 학습을 시작합니다.  
이렇게 하면 학습 루프가 시작되고 3 에폭 동안 모델이 학습됩니다. PEFT 방법을 사용하고 있기 때문에 전체 모델이 아닌 조정된 모델 가중치만 저장할 예정입니다.

In [19]:
# 학습 시작, 모델은 자동으로 허브와 출력 디렉토리에 저장됨
trainer.train()

# 모델 저장
trainer.save_model(args.output_dir)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Step,Training Loss
5,2.222200
10,1.823000
15,1.294200
20,0.904500
25,0.679900
30,0.638500
35,0.576900
40,0.538800
45,0.586400
50,0.550700


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Unrecognized keys in `rope_scaling` for 'r

In [20]:
# 메모리 비우기
del model
del trainer
torch.cuda.empty_cache()

## 인퍼런스

학습이 완료된 후에는 모델을 평가하고 테스트해볼 예정입니다.  

먼저 기본 모델을 불러와서 임의의 아마존 제품에 대한 설명을 생성해보고, 그 다음 Q-LoRA로 조정된 모델을 불러와 같은 제품에 대한 설명을 생성해볼 것입니다.  
마지막으로 더 효율적인 추론을 위해 어댑터를 기본 모델과 병합한 뒤, 동일한 제품에 대해 다시 한 번 추론을 실행해볼 예정입니다.

In [21]:
model_id = "Qwen/Qwen2-VL-72B-Instruct" 

In [22]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
 
# 기본 모델 호출
model = AutoModelForVision2Seq.from_pretrained(
  model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(model_id)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

In [23]:
# 모델 답변을 생성하는 함수
def generate_description(messages, model, processor):
   # 추론을 위한 준비
   text = processor.apply_chat_template(
       messages, tokenize=False, add_generation_prompt=True
   )
   image_inputs, video_inputs = process_vision_info(messages)
   inputs = processor(
       text=[text],
       images=image_inputs,
       videos=video_inputs,
       padding=True,
       return_tensors="pt",
   )
   inputs = inputs.to(model.device)
   # 추론: 출력 생성
   generated_ids = model.generate(**inputs, max_new_tokens=128, top_p=1.0, do_sample=True, temperature=0.1)
   generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
   output_text = processor.batch_decode(
       generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
   )
   return output_text[0]

In [24]:
# 설명 생성해보기
messages =  test_dataset[0]["messages"]
base_description = generate_description(messages, model, processor)
print(base_description)
# 원하는 경우 아래 명령어로 활성화된 어댑터를 비활성화할 수 있음
# model.disable_adapters()

소금물에서 더 잘 뜨나요.


In [25]:
# 로라 어댑터가 있는 경로
adapter_path = "./qwen2-7b-instruct-homeworks"

In [26]:
model.load_adapter(adapter_path) 

In [27]:
ft_description = generate_description(messages, model, processor)
print(ft_description)

소금물에서 더 잘 뜨나요?


In [28]:
print('LoRA 학습 전 모델:', base_description)
print('---')
print('LoRA 학습 후 모델:', ft_description)

LoRA 학습 전 모델: 소금물에서 더 잘 뜨나요.
---
LoRA 학습 후 모델: 소금물에서 더 잘 뜨나요?


In [29]:
from tqdm.auto import tqdm 
result = [] 
for idx in tqdm(range(len(test_dataset))):
    messages =  test_dataset[idx]["messages"]
    ft_description = generate_description(messages, model, processor)
    result.append(ft_description)

  0%|          | 0/622 [00:00<?, ?it/s]

In [30]:
result[:10]

['소금물에서 더 잘 뜨나요?',
 '두 쌍의 자기력 크기는 같습니다.',
 '매사추세츠',
 '피닉스',
 '용액 B',
 '델라웨어',
 '샘플 B',
 '메릴랜드',
 '호주',
 '끌어당기다']

In [31]:
answer = [data["messages"][2]["content"][0]["text"] for data in test_dataset]
answer[:10]

['계란이 민물에서 더 잘 뜨나요, 아니면 소금물에서 더 잘 뜨나요?',
 '두 쌍의 자기력 크기는 같습니다.',
 '매사추세츠',
 '피닉스',
 '용액 B',
 '델라웨어',
 '샘플 B',
 '켄터키',
 '호주',
 '밀어내다']

In [35]:
484/662*100, 286/662*100

(73.1117824773414, 43.202416918429)

In [38]:
score = 0 
for a, b in zip(result, answer):
    if a == b : 
        score += 1
score 

484

## 로라 병합 후 저장하는 코드

In [ ]:
from peft import PeftModel
from transformers import AutoProcessor, AutoModelForVision2Seq

adapter_path = "./qwen2-7b-instruct-homeworks"  # 학습된 어댑터 경로
base_model_id = "Qwen/Qwen2-VL-72B-Instruct"  # 기본 모델 ID
merged_path = "merged"  # 병합된 모델을 저장할 경로

# 기본 모델 로드
model = AutoModelForVision2Seq.from_pretrained(model_id, low_cpu_mem_usage=True)

# 병합된 모델 저장 경로
# LoRA와 기본 모델을 병합하고 저장
peft_model = PeftModel.from_pretrained(model, adapter_path)  # PEFT 모델 로드
merged_model = peft_model.merge_and_unload()  # 모델 병합
merged_model.save_pretrained(merged_path,safe_serialization=True, max_shard_size="2GB")  # 병합된 모델 저장

processor = AutoProcessor.from_pretrained(base_model_id)  # 프로세서 로드
processor.save_pretrained(merged_path)  # 프로세서 저장

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

In [4]:
from peft import PeftModel
from transformers import AutoProcessor, AutoModelForVision2Seq
model_id = "Qwen/Qwen2-VL-72B-Instruct" 
adapter_path = "./qwen2-7b-instruct-homeworks"  # 학습된 어댑터 경로
base_model_id = "Qwen/Qwen2-VL-72B-Instruct"  # 기본 모델 ID
merged_path = "merged"  # 병합된 모델을 저장할 경로

# 기본 모델 로드
model = AutoModelForVision2Seq.from_pretrained(model_id, low_cpu_mem_usage=True)

# 병합된 모델 저장 경로
# LoRA와 기본 모델을 병합하고 저장
peft_model = PeftModel.from_pretrained(model, adapter_path)  # PEFT 모델 로드
merged_model = peft_model.merge_and_unload()  # 모델 병합
merged_model.save_pretrained(merged_path,safe_serialization=True, max_shard_size="2GB")  # 병합된 모델 저장

processor = AutoProcessor.from_pretrained(base_model_id)  # 프로세서 로드
processor.save_pretrained(merged_path)  # 프로세서 저장

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/108k [00:00<?, ?B/s]

model-00001-of-00038.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00038.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

[]

In [5]:
print("모두 완료했습니다")

모두 완료했습니다


In [7]:
local_model_path = "/workspace/trl/merged"
model = AutoModelForVision2Seq.from_pretrained(local_model_path)
processor = AutoProcessor.from_pretrained(local_model_path)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/164 [00:00<?, ?it/s]

In [8]:
!huggingface-cli login --token Your_Huggingface_Token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingface` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingface`


In [ ]:
hub_model_id = "daje/Qwen2-VL-72B-instruct-ScienceQA"
# 허깅페이스 업로드 
model.push_to_hub(hub_model_id)
processor.push_to_hub(hub_model_id)